# Test FSI Problem with Different Models



In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
import os
import torch
import numpy as np
import gc

import h5py
import pandas as pd
from matplotlib import pyplot as plt

Import local packages

In [13]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../.."))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.utils.logger import Logging
from src.utils.utils import lp_error
from src.utils.logger import Logging

from src.utils.plot_loss import plot_loss_history

Set various constant variables: model path, name, etc.

# Create logger

In [14]:
logger = Logging(TEST_CHECKPOINT_PATH)
model_dirname = logger.get_output_dir()

In [16]:
from src.data.IBM_data_loader import generate_fluid_testing_dataset

TEST_DATA_PATH = "../../../data/IB_PINN3.mat"


[time_, xfa, yfa, ufa, vfa, pfa, fxfa, fyfa] = generate_fluid_testing_dataset(
    TEST_DATA_PATH
)


input_ = torch.tensor(
    np.concatenate(
        [time_, xfa, yfa],
        axis=1,
    ),
    dtype=torch.float32,
)

In [25]:
TEST_DATA_PKL = "../../data/IB_PINN3.mat"
TEST_CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "result/fsi")

RBA = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_18-41-49-625964/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-36-49-818397/model.pth"
FIXED_WEIGHT = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_21-46-08-833958/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-11-45-691256/model.pth"
SA = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_18-42-12-444785/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-33-081291/model.pth"
Grad_stat = "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_18-41-55-608353/model.pth"  # "/home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_17-37-23-782676/model.pth"

MODEL_PATH_LIST = {
    # "RBA": RBA,
    "Fixed": FIXED_WEIGHT,
    # "SA": SA,
    # "grad_stat": Grad_stat,
}


SOLVER_TO_MODULE = {
    "tanh": "src.nn.tanh",
    "xsig": "src.nn.xsigmoid",
}

## Load the Model and Test

In [26]:
for method, model_path in MODEL_PATH_LIST.items():
    logger.print(f"MODEL_PATH {model_path}")
    # Load the state from the saved model
    state = torch.load(
        model_path,
    )
    config = state.get("config", {})
    solver = config.get("solver")

    # Extract model configuration from state
    model_activation_name = config.get("activation", "Tanh")
    network_fluid = config.get("network_fluid")
    network_force = config.get("network_force")
    loss_history = state.get("loss_history")

    data_mean = state.get("data_mean").to("cpu")
    data_std = state.get("data_std").to("cpu")

    # Dynamically import the correct module and class
    if solver in SOLVER_TO_MODULE:
        module = __import__(SOLVER_TO_MODULE[solver], fromlist=["PINNKAN"])
        PINNKAN = getattr(module, "PINNKAN")

        # Initialize fluid and solid models
        model_fluid = PINNKAN(network_fluid, model_activation_name).to("cpu")
        model_force = PINNKAN(network_force, model_activation_name).to("cpu")

    model_fluid.load_state_dict(state["model_fluid_state_dict"])
    model_force.load_state_dict(state["model_force_state_dict"])
    logger.print(f"model loaded from {model_path}")
    logger.print(f"problem: {config.get('problem')}")
    logger.print(f"dataset_path:  , {config.get('dataset_path')}")
    logger.print(f"batch size:  , {config.get('batch_size')}")
    logger.print(f"network:  , {config.get('network')}")
    logger.print(f"term loss weights :  , {config.get('weights')}")
    logger.print(f"method:  , {config.get('weighting')}")
    logger.print(f"solver:  , {config.get('solver')}")
    logger.print(
        f"number of iterations:  , {len(loss_history[next(iter(loss_history))])}"
    )

    test_torch_data = torch.tensor(
        np.concatenate([time_, xfa, yfa], axis=1), dtype=torch.float32
    ).to("cpu")
    with torch.no_grad():
        predictions1 = model_fluid.forward(test_torch_data, data_mean, data_std)
        predictions2 = model_force.forward(test_torch_data, data_mean, data_std)
    if predictions1.is_cuda:
        predictions1 = predictions1.cpu()
        predictions2 = predictions2.cpu()
    u_pred = predictions1[:, 0:1].numpy()
    v_pred = predictions1[:, 1:2].numpy()
    p_pred = predictions1[:, 2:3].numpy()
    fx_pred = predictions2[:, 0:1].numpy()
    fy_pred = predictions2[:, 1:2].numpy()

    text = "RelL2_"
    # logger.print("\n Relative L2 ERROR:")
    u_error2 = lp_error(u_pred, ufa, (text + "U%"), logger, 2)
    v_error2 = lp_error(v_pred, vfa, (text + "V%"), logger, 2)
    p_error2 = lp_error(p_pred, pfa, (text + "P%"), logger, 2)
    fx_error2 = lp_error(fx_pred, fxfa, (text + "Fx%"), logger, 2)
    fx_error2 = lp_error(fy_pred, fxfa, (text + "Fx%"), logger, 2)

    for key in loss_history:
        logger.print("Final loss %s: %e" % (key, loss_history[key][-1]))

    final_loss_sum = sum([l[-1] for l in loss_history.values()])
    logger.print(f"final loss sum: {final_loss_sum:.2e}")

    logger.print("******************************\n")

    # Delete model and clear cache after each iteration
    del model_fluid
    del model_force
    torch.cuda.empty_cache()
    gc.collect()  # Force garbage collection to release memory

logger.print("file directory:", logger.get_output_dir())

INFO:src.utils.logger:MODEL_PATH /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_21-14-36-711081/model.pth
INFO:src.utils.logger:model loaded from /home/ubuntu/afrah/code/pinn_fsi_ibm/checkpoints/fsi/2024-10-15_21-14-36-711081/model.pth
INFO:src.utils.logger:problem: fsi
INFO:src.utils.logger:dataset_path:  , None
INFO:src.utils.logger:batch size:  , 100
INFO:src.utils.logger:network:  , None
INFO:src.utils.logger:term loss weights :  , None
INFO:src.utils.logger:method:  , Fixed
INFO:src.utils.logger:solver:  , xsig
INFO:src.utils.logger:number of iterations:  , 20001
INFO:src.utils.logger:RelL2_U%  : 82.88 
INFO:src.utils.logger:RelL2_V%  : 112.87 
INFO:src.utils.logger:RelL2_P%  : 123.94 
INFO:src.utils.logger:RelL2_Fx%  : 11666.85 
INFO:src.utils.logger:RelL2_Fx%  : 9673.82 
INFO:src.utils.logger:Final loss left: 8.032391e-01
INFO:src.utils.logger:Final loss right: 8.147967e-01
INFO:src.utils.logger:Final loss bottom: 7.908438e-01
INFO:src.utils.logger:Final loss up